In [1]:
import numpy as np
import glob
from scipy.optimize import root_scalar

In [5]:
cfs = glob.glob('complete_stream/*.txt')

In [6]:
for file in cfs:

    with open(file, 'r') as in_file:
        noz_length = float(in_file.readlines()[1].replace('#', '').strip())

    ar = np.genfromtxt(file, delimiter=',', skip_header=2)
    throat = ar[0, 0]
    reduced_ar = ar[ar[:, 0] >= throat, 0:2]
    reduced_ar = reduced_ar[reduced_ar[:, 0] <= 1.5 * throat]
    reduced_ar[:, 0] -= throat
    _, validIdx = np.unique(reduced_ar[:, 0], return_index=True)
    reduced_ar = reduced_ar[validIdx, :]
    reduced_ar = np.hstack([reduced_ar, np.zeros((reduced_ar.shape[0], 1))])

    rc = 6

    r = rc * reduced_ar[0, 1]
    y0 = reduced_ar[0, 1]

    x1 = reduced_ar[1, 0]
    y1 = reduced_ar[1, 1]

    def interpolate_between(x1, x2, y1, y2, dydx1, dydx2, npts=21):
        pow0 = np.array([3, 2, 1, 0])
        pow1 = np.array([2, 1, 0, 0])

        k0 = np.array([1, 1, 1, 1])
        k1 = np.array([3, 2, 1, 0])

        A = np.array([k0 * x1 ** pow0, k1 * x1 ** pow1, k0 * x2 ** pow0, k1 * x2 ** pow1])

        B = np.array([y1, s1, y2, s2])

        coeffs = np.linalg.solve(A, B)

        x_vals = np.linspace(x1, x2, 9)
        y_vals = np.sum(coeffs * x_vals[:, None] ** pow0, axis=1)

        return x_vals, y_vals

    valid_ar = np.logical_and(ar[:, 0] >= ar[0, 0], ar[:, 0] <= ar[0, 0] + noz_length)
    the_ar = ar[valid_ar, :]
    _, unique_idx = np.unique(the_ar[:, 0], return_index=True)
    the_ar = the_ar[unique_idx]

    this_idx = 0

    x1 = the_ar[this_idx, 0]
    y1 = the_ar[this_idx, 1]
    s1 = the_ar[this_idx, 2]

    x2 = the_ar[this_idx + 1, 0]
    y2 = the_ar[this_idx + 1, 1]
    s2 = the_ar[this_idx + 1, 2]

    x_val_ar = []
    y_val_ar = []

    for x1, y1, s1, x2, y2, s2 in zip(the_ar[:-1, 0], the_ar[:-1, 1], the_ar[:-1, 2], the_ar[1:, 0], the_ar[1:, 1], the_ar[1:, 2]):
        x_vals, y_vals = interpolate_between(x1, x2, y1, y2, s1, s2)
        x_val_ar.append(x_vals)
        y_val_ar.append(y_vals)

    x_vals = np.hstack(x_val_ar) - x_val_ar[0][0]
    y_vals = np.hstack(y_val_ar)
    xyz_vals = np.vstack([x_vals, y_vals, np.zeros(x_vals.shape)]).T
    
    _, unique_idx = np.unique(x_vals, return_index=True)
    
    file_short = file.split('\\')[1]
    np.savetxt(f"SolidWorks_stream/{file_short}", xyz_vals[unique_idx, :])